In [1]:
from dataset import read_raw, get_subjects, get_path, mne_events
from utils import decod_xy
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy
nlp = spacy.load("fr_core_news_sm")

all_evos = []
all_scores = []

path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
# Debug
runs = 1

epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}


            
for subject in subjects[2:3]:
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    # Dict init
    for start in ('onset', 'offset'): 
            for level in ('word', 'constituent', 'sentence'):
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True)
        meta = meta_.copy()
        # Metadata update
        # Word start
        meta['word_onset'] = True

        # Word end
        meta['word_offset'] = True

        # Sent start
        meta['sentence_onset'] = meta.word_id == 0

        # Sent stop
        meta['next_word_id'] = meta['word_id'].shift(-1)
        meta['sentence_offset'] = meta.apply(lambda x: True if x['word_id'] > x['next_word_id'] else False, axis=1)
        meta['sentence_offset'].fillna(False, inplace=True)
        meta.drop('next_word_id', axis=1, inplace=True)

        # Const start
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: True if x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1 else False, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)

        # Const stop
        meta['next_closing'] = meta['n_closing'].shift(-1)
        meta['constituent_offset'] = meta.apply(lambda x: True if x['n_closing'] > x['next_closing'] else False, axis=1)
        meta['constituent_offset'].fillna(False, inplace=True)
        meta.drop('next_closing', axis=1, inplace=True)

        for start in ('onset', 'offset'): 
            # for level in ('word', 'constituent', 'sentence'):
            for level in ('sentence', 'constituent', 'word'):
                # Select only the rows containing the True for the conditions (sentence_end, etc..)
                sel = meta.query(f'{level}_{start}==True')
                assert sel.shape[0] > 10  #
                # TODO check variance as well for sentences
                # Matchlist events and meta
                # So that we can epoch now that's we've sliced our metadata
                i, j = match_list(events[:, 2], sel.word.apply(len))
                sel = sel.reset_index().loc[j]
                epochs = mne.Epochs(raw, **mne_events(sel, raw), decim = 10,
                                     tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                       tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                         event_repeated = 'drop', # check event repeated
                                            preload=True)  # n_words OR n_constitutent OR n_sentences
                epoch_key = f'{level}_{start}'
            
                dict_epochs[epoch_key].append(epochs)
        
            
    # Once we have the dict of epochs per condition full (epoching for each run for a subject)
    # we can concatenate them, and fix the dev_head             
    for start_ in ('onset', 'offset'): 
        for level_ in ('word', 'constituent', 'sentence'):
            epoch_key = f'{level_}_{start_}'
            all_epochs_chosen = dict_epochs[epoch_key]
            # Concatenate epochs
            for epo in all_epochs_chosen:
                epo.info["dev_head_t"] = all_epochs_chosen[1].info["dev_head_t"]

            dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)

    # Now that we have all the epochs, rerun the plotting / decoding on averaged
    for start in ('onset', 'offset'): 
            for level in ('word', 'constituent', 'sentence'):  
                epoch_key = f'{level}_{start}'
                epochs = dict_epochs[epoch_key]
                # mean
                evo = epochs.copy().pick_types(meg=True).average(method='median')
                all_evos.append(dict(subject=subject, evo=evo, start=start, level=level))

all_scores = pd.DataFrame(all_scores,index=False)
all_evos = pd.DataFrame(all_evos,index=False)

all_scores.to_csv('./score.csv')
all_evos.to_csv('./evos.csv')



 Epoching for run 1, subject: 3

Opening raw data file /home/is153802/data/LPP_MEG_visual/sub-3/ses-01/meg/sub-3_ses-01_task-read_run-01_meg.fif...
    Read a total of 13 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v6 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v8 (1 x 306)  idle
    Range : 36000 ... 517999 =     36.000 ...   517.999 secs
Ready.
Reading events from /home/is153802/data/LPP_MEG_

/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:51: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:51: RuntimeWarning: Omitted 81 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:51: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


Trigger channel has a non-zero initial value of 8 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
1466 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
            word  n_closing  is_last_word      pos
0        lorsque          1         False       CS
1              j          1         False      XXX
2          avais          1         False  CLS-SUJ
3            six          1         False      DET
4           ans,          4         False       NC
5              j          1         False      XXX
6             ai          1         False  CLS-SUJ
7            vu,          2         False      VPP
8            une          1         False      DET
9          fois,          2         False       NC
10           une          1         False      DET
11    magnifique          1         False      ADJ
12        image,          2         False       NC
13          dans          1         False       

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:    6.7s finished


Adding metadata with 21 columns
38 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 38 events and 5001 original time points (prior to decimation) ...


...

/tmp/ipykernel_142831/1277334967.py:76: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, **mne_events(sel, raw), decim = 10,
/tmp/ipykernel_142831/1277334967.py:76: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, **mne_events(sel, raw), decim = 10,


1 bad epochs dropped
Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Adding metadata with 21 columns
1405 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1405 events and 1301 original time points (prior to decimation) ...


/tmp/ipykernel_142831/1277334967.py:76: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, **mne_events(sel, raw), decim = 10,


1 bad epochs dropped


IndexError: list index out of range